# Questionnaire Processing

This Notebook processes questionnaire data and extracts relevant columns. The informaton used from the questionnaire data are:
* Chronotype: assessed by *Morningness-Eveningness Questionnaire (MEQ)*
* Sleep Information: Self-reported Bed Time, Sleep Onset, Wake Onset

As Questionnaire for Chronotype Assessment we use the Morningness Eveningness Questionnaire (MEQ) from Horne and Östberg (1976).

```
Horne, J. A., & Östberg, O. (1976). A self-assessment questionnaire to determine morningness-eveningness in human circadian rhythms. International journal of chronobiology.
```

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.questionnaires.utils import invert, find_cols
from biopsykit.utils.dataframe_handling import int_from_str_idx, camel_to_snake

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
sns.set(style='ticks')

## Load Questionnaire Data

In [ ]:
data_path = Path("../../../../Data")
export_path = Path("../..").joinpath("exports")

bp.utils.file_handling.mkdirs(export_path)

In [ ]:
df_quest = bp.io.load_questionnaire_data(data_path.joinpath("Questionnaire_Data").joinpath("Questionnaire_Data_CARWatch.xlsx"), index_cols='subject')
df_quest.head()

## Chronotype

### Convert MEQ Questionnaire Items Into Right Format

In [ ]:
df_meq = find_cols(df_quest, starts_with="MEQ", ends_with="[0-9]")[0]

# Recode MEQ01
df_meq.loc[:, 'MEQ_01'].replace({1: 1, 2: 1, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}, inplace=True)
# Recode MEQ02
df_meq.loc[:, 'MEQ_02'].replace({1: 1, 2: 1, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}, inplace=True)
# Recode MEQ10
df_meq.loc[:, 'MEQ_10'].replace({1: 1, 2: 1, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}, inplace=True)

# Invert columns that were in inverted order in questionnaire (to comply with biopsykit implementation)
invert_cols = ['MEQ_03', 'MEQ_08', 'MEQ_09', 'MEQ_19']
invert(df_meq.loc[:, invert_cols], score_range=[1, 4], inplace=True)

# Invert columns that were in inverted order in questionnaire (to comply with biopsykit implementation)
invert_cols = ['MEQ_17', 'MEQ_18']
invert(df_meq.loc[:, invert_cols], score_range=[1, 5], inplace=True)

meq = bp.questionnaires.meq(df_meq)

### Compute MEQ and Classify Chronotypes

From the MEQ score Chronotypes can be classified in two different ways:
* Fine Classification (5 levels, column `Chronotype_Fine`):
    - 0: definite evening type (MEQ score 14-30)
    - 1: moderate evening type (MEQ score 31-41)
    - 2: intermediate type (MEQ score 42-58)
    - 3: moderate morning type (MEQ score 59-69)
    - 4: definite morning type (MEQ score 70-86)
* Coarse Classification (3 levels, column `Chronotype_Coarse`):
    - 0: evening type (MEQ score 14-41)
    - 1: intermediate type (MEQ score 42-58)
    - 2: morning type (MEQ score 59-86)

In [ ]:
meq.head()

### Further Information

#### MEQ Histogram

In [ ]:
fig, ax = plt.subplots()
meq['MEQ'].plot(kind='hist', ax=ax)
ax.axvline(41, color='grey', ls='--')
ax.axvline(58, color='grey', ls='--')
ax.set_xlabel("MEQ Score")
ax.set_ylabel("Count")

#### Chronotype Prevalence

In [ ]:
pd.DataFrame(meq['Chronotype_Coarse'].value_counts())

In [ ]:
meq.describe().T

## Sleep Information

### Ideal Bedtime Ranges

In [ ]:
bedtime_ranges = {1: ["01:45:00", "03:00:00"], 2: ["00:30:00", "01:45:00"], 3: ["22:15:00", "00:30:00"], 4: ["21:00:00", "22:15:00"], 5: ["20:00:00", "21:00:00"]}

bedtime_ranges = pd.DataFrame(bedtime_ranges, index=["start", "end"]).T
bedtime = pd.DataFrame({
    'ideal_bed_{}'.format(key): df_meq['MEQ_02'].replace(bedtime_ranges[key])
    for key in ['start', 'end']
})

### Self-Report Sleep Data

In [ ]:
times_selfreport = df_quest.filter(regex="(bed|sleepOnset|wakeOnset)Selfreport_*")

## Merge Data

In [ ]:
df_quest_sleep = pd.concat([bedtime, meq, times_selfreport], axis=1)
df_quest_sleep.head()

### Convert To Long-Format

In [ ]:
df_quest_sleep = pd.wide_to_long(df_quest_sleep.reset_index(), stubnames=['{}Selfreport'.format(s) for s in ['sleepOnset', 'bed', 'wakeOnset']], i="subject", j="night", sep="_", suffix='\w+').sort_index()
df_quest_sleep = int_from_str_idx(df_quest_sleep, 'night', "N(\w)", lambda x: x-1)
df_quest_sleep.head()

In [ ]:
df_quest_sleep = df_quest_sleep.rename(columns={s: camel_to_snake(s) for s in ["sleepOnsetSelfreport", "bedSelfreport", "wakeOnsetSelfreport"]})
df_quest_sleep = df_quest_sleep.rename(columns={s: s.lower() for s in ["Chronotype_Coarse", "Chronotype_Fine"]})
df_quest_sleep.head()

In [ ]:
df_quest_sleep.to_csv(export_path.joinpath("questionnaire_chronotype_bedtimes.csv"))